[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Isotr0py/SakuraLLM-Notebooks/blob/main/Sakura-13B-Galgame-Colab.ipynb)

In [1]:
#@title 初始化环境
#@markdown 挂载Google网盘
Mount_GDrive = False # @param {type:"boolean"}
if Mount_GDrive:
  from google.colab import drive

  drive.mount('/content/gdrive')
  ROOT_PATH = "/content/gdrive/MyDrive"
else:
  ROOT_PATH = "/content"
!nvidia-smi

Fri Sep  6 02:46:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#@title 安装依赖
%cd $ROOT_PATH
!git clone https://github.com/SakuraLLM/Sakura-13B-Galgame.git

%cd Sakura-13B-Galgame
!git pull

LLAMA_CPP = True # @param {type:"boolean"}
VLLM = True # @param {type:"boolean"}
if LLAMA_CPP:
  !pip install "diskcache>=5.6.1"
  !pip install llama-cpp-python -i https://abetlen.github.io/llama-cpp-python/whl/cu122
if VLLM:
  !pip install -U transformers tokenizers
  !pip install vllm
!pip install -q -r requirements.txt
!pip install -q pyngrok

In [ ]:
#@title 翻译EPUB
from huggingface_hub import hf_hub_download

repo_id = "SakuraLLM/Sakura-14B-Qwen2beta-v0.9.2-GGUF"
MODEL = "sakura-14b-qwen2beta-v0.9.2-q4km.gguf" # @param ["sakura-14b-qwen2beta-v0.9.2-iq4xs.gguf", "sakura-14b-qwen2beta-v0.9.2-q2k.gguf", "sakura-14b-qwen2beta-v0.9.2-q3km.gguf", "sakura-14b-qwen2beta-v0.9.2-q4km.gguf", "sakura-14b-qwen2beta-v0.9.2-q6k.gguf"]
hf_hub_download(repo_id=repo_id, filename=MODEL, local_dir="models/")
MODEL_PATH = f"./models/{MODEL}"
EPUB_PATH = "novel.epub" # @param {type:"string"}
GPT_DICT_PATH = "dict.txt"  # @param {type:"string"}
OUTPUT_FOLDER = "output/" # @param {type:"string"}

%cd $ROOT_PATH/Sakura-13B-Galgame
!python translate_epub.py \
    --model_name_or_path $MODEL_PATH \
    --llama_cpp \
    --use_gpu \
    --model_version 0.9 \
    --trust_remote_code \
    --data_path $EPUB_PATH \
    --gpt_dict_path $GPT_DICT_PATH \
    --output_folder $OUTPUT_FOLDER

In [ ]:
#@title 翻译文本文件
from huggingface_hub import hf_hub_download

repo_id = "SakuraLLM/Sakura-14B-Qwen2beta-v0.9.2-GGUF"
MODEL = "sakura-14b-qwen2beta-v0.9.2-q4km.gguf" # @param ["sakura-14b-qwen2beta-v0.9.2-iq4xs.gguf", "sakura-14b-qwen2beta-v0.9.2-q2k.gguf", "sakura-14b-qwen2beta-v0.9.2-q3km.gguf", "sakura-14b-qwen2beta-v0.9.2-q4km.gguf", "sakura-14b-qwen2beta-v0.9.2-q6k.gguf"]
hf_hub_download(repo_id=repo_id, filename=MODEL, local_dir="models/")
MODEL_PATH = f"./models/{MODEL}"
DATA_PATH = "novel.txt" # @param {type:"string"}
OUTPUT_PATH = "novel_translated.txt" # @param {type:"string"}

%cd $ROOT_PATH/Sakura-13B-Galgame
!python translate_novel.py \
    --model_name_or_path $MODEL_PATH \
    --llama_cpp \
    --use_gpu \
    --model_version 0.9 \
    --trust_remote_code \
    --data_path $DATA_PATH \
    --output_path $OUTPUT_PATH

In [ ]:
#@title 运行API后端(vLLM) **🚧暂不可用🚧**
#@markdown 使用[ngrok](https://ngrok.com/)进行API映射（ngrokToken留空则使用[localtunnel](https://github.com/localtunnel/localtunnel)）
from huggingface_hub import hf_hub_download

ngrokToken = ""  # @param {type:"string"}
if ngrokToken:
  from pyngrok import conf, ngrok
  conf.get_default().auth_token = ngrokToken
  conf.get_default().monitor_thread = False
  ssh_tunnels = ngrok.get_tunnels(conf.get_default())
  if len(ssh_tunnels) == 0:
      ssh_tunnel = ngrok.connect(5000)
      print('address：'+ssh_tunnel.public_url)
  else:
      print('address：'+ssh_tunnels[0].public_url)
else:
  import subprocess
  import threading
  def start_localtunnel(port):
      p = subprocess.Popen(["npx", "localtunnel", "--port", f"{port}"], stdout=subprocess.PIPE)
      for line in p.stdout:
          print(line.decode(), end='')
  threading.Thread(target=start_localtunnel, daemon=True, args=(5000,)).start()


%cd $ROOT_PATH/Sakura-13B-Galgame

repo_id = "SakuraLLM/Sakura-14B-Qwen2beta-v0.9.2-GGUF"
MODEL = "sakura-14b-qwen2beta-v0.9.2-q4km.gguf" # @param ["sakura-14b-qwen2beta-v0.9.2-iq4xs.gguf", "sakura-14b-qwen2beta-v0.9.2-q2k.gguf", "sakura-14b-qwen2beta-v0.9.2-q3km.gguf", "sakura-14b-qwen2beta-v0.9.2-q4km.gguf", "sakura-14b-qwen2beta-v0.9.2-q6k.gguf"]
hf_hub_download(repo_id=repo_id, filename=MODEL, local_dir="models/")
MODEL_PATH = f"./models/{MODEL}"

!RAY_memory_monitor_refresh_ms="0" python server.py \
  --model_name_or_path $MODEL_PATH \
  --vllm \
  --model_version 0.9 \
  --trust_remote_code \
  --no-auth \
  --enforce_eager \
  --gpu_memory_utilization 0.95

In [ ]:
#@title 运行API后端(llama-cpp-python)
#@markdown 使用[ngrok](https://ngrok.com/)进行API映射（ngrokToken留空则使用[localtunnel](https://github.com/localtunnel/localtunnel)）
from huggingface_hub import hf_hub_download

ngrokToken = ""  # @param {type:"string"}
if ngrokToken:
  from pyngrok import conf, ngrok
  conf.get_default().auth_token = ngrokToken
  conf.get_default().monitor_thread = False
  ssh_tunnels = ngrok.get_tunnels(conf.get_default())
  if len(ssh_tunnels) == 0:
      ssh_tunnel = ngrok.connect(5000)
      print('address：'+ssh_tunnel.public_url)
  else:
      print('address：'+ssh_tunnels[0].public_url)
else:
  import subprocess
  import threading
  def start_localtunnel(port):
      p = subprocess.Popen(["npx", "localtunnel", "--port", f"{port}"], stdout=subprocess.PIPE)
      for line in p.stdout:
          print(line.decode(), end='')
  threading.Thread(target=start_localtunnel, daemon=True, args=(5000,)).start()


%cd $ROOT_PATH/Sakura-13B-Galgame

repo_id = "SakuraLLM/Sakura-14B-Qwen2beta-v0.9.2-GGUF"
MODEL = "sakura-14b-qwen2beta-v0.9.2-iq4xs.gguf" # @param ["sakura-14b-qwen2beta-v0.9.2-iq4xs.gguf", "sakura-14b-qwen2beta-v0.9.2-q2k.gguf", "sakura-14b-qwen2beta-v0.9.2-q3km.gguf", "sakura-14b-qwen2beta-v0.9.2-q4km.gguf", "sakura-14b-qwen2beta-v0.9.2-q6k.gguf"]
hf_hub_download(repo_id=repo_id, filename=MODEL, local_dir="models/")
MODEL_PATH = f"./models/{MODEL}"

!python server.py \
  --model_name_or_path $MODEL_PATH \
  --llama_cpp \
  --use_gpu \
  --model_version 0.9 \
  --trust_remote_code \
  --no-auth